In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler, Adam
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torchvision
import torchvision.utils as vutils
import cv2
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import torch.nn.init as init

import pickle

trans= transforms.Compose([transforms.Resize((64,64)),
                            transforms.ToTensor(),
                          transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset= torchvision.datasets.ImageFolder(root='D:/nodejshm/public/SignImages', transform=trans)

trainloader= DataLoader(trainset, batch_size=6, shuffle= False, num_workers=2)
train_loader= DataLoader(trainset, batch_size=6, shuffle= False, num_workers=2)
dataloader= DataLoader(trainset, batch_size=6, shuffle= False, num_workers=2)

dataiter= iter(trainloader)
images, labels= dataiter.next()

def imshow(img):
    img = img/ 2 +0.5
    np_img= img.numpy()
    #plt.imshow(np_img)
    plt.imshow(np.transpose(np_img, (1,2,0)))
    
    print(np_img.shape)
    print((np.transpose(np_img, (1,2,0))).shape)
    
#for n, (img, labels) in enumerate(trainloader):
    #print(n, img.shape, labels.shape)

#print(''.join('%5s' %labels[j] for j in range(6)))
#print(images.shape)
#print((torchvision.utils.make_grid(images)).shape)
#imshow(torchvision.utils.make_grid(images, nrow=6))

In [3]:
nz     = 100      #dimension of noise vector
nc     = 3        #channel
ngf    = 64       #generator 레이어 필터 개수
ndf    = 64       #discriminator 레이어 필터 개수
niter  = 2000     #전체 number epoch
lr     = 0.0002   #learning rate
beta1  = 0.5      #hyper parameter of Adam optimizer
ngpu   = 1        #GPU

imageSize = 64    
batchSize = 6    

outf = "D:/nodejshm/public/SignImages/test02"

In [4]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:         
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1: 
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [5]:
class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf* 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf* 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf* 8, ngf* 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf* 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf* 4, ngf* 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf* 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf* 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu> 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [6]:
class _netD(nn.Module):
    def __init__(self, ngpu):
        super(_netD, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output.view(-1, 1).squeeze(1)

In [7]:
netG = _netG(ngpu)
netG.apply(weights_init)
print(netG)

netD = _netD(ngpu)
netD.apply(weights_init)
print(netD)

_netG(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
_netD(
  (main): Sequent

In [8]:
criterion = nn.BCELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

netD.cuda()
netG.cuda()
criterion.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

In [9]:
fixed_noise = Variable(fixed_noise)

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [10]:
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        #Update D network: maximize log(D(x))+ log(1-D(G(z)))
        
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)

        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()

        errD = errD_real+ errD_fake
        optimizerD.step()
        #Update G network: maximize log(D(G(z)))

        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))
        output = netD(fake)

        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

    vutils.save_image(real_cpu, '%s/real_samples.png'% outf, normalize=True)
    fake = netG(fixed_noise)
    vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png'% (outf, epoch), normalize=True)
    
    print('<%d/%d##%d/%d> <Loss_D: %.4f> <Loss_G: %.4f> <D(x): %.4f> D(G(z12)): %.4f//%.4f'
      % (epoch, niter, i, len(dataloader), errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
    loss_D.append(errD.data[0])
    loss_G.append(errG.data[0])
    score_D.append(D_x)
    score_G1.append(D_G_z1)
    score_G2.append(D_G_z2)
    result_dict = {"loss_D":loss_D,"loss_G":loss_G,"score_D":score_D,"score_G1":score_G1,"score_G2":score_G2}
    pickle.dump(result_dict,open("{}/result_dict.p".format(outf),"wb"))

    torch.save(netG.state_dict(), '%s/netG.pth' % (outf))
    torch.save(netD.state_dict(), '%s/netD.pth' % (outf))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


<0/2000##0/1> <Loss_D: 1.0885> <Loss_G: 8.7732> <D(x): 0.6073> D(G(z12)): 0.4006//0.0002
<1/2000##0/1> <Loss_D: 0.5836> <Loss_G: 9.8017> <D(x): 0.9997> D(G(z12)): 0.4091//0.0001
<2/2000##0/1> <Loss_D: 0.0970> <Loss_G: 7.4519> <D(x): 0.9999> D(G(z12)): 0.0873//0.0008
<3/2000##0/1> <Loss_D: 0.0245> <Loss_G: 6.5270> <D(x): 0.9999> D(G(z12)): 0.0239//0.0018
<4/2000##0/1> <Loss_D: 0.0818> <Loss_G: 6.4761> <D(x): 0.9999> D(G(z12)): 0.0770//0.0018
<5/2000##0/1> <Loss_D: 0.0671> <Loss_G: 6.5341> <D(x): 0.9998> D(G(z12)): 0.0639//0.0015
<6/2000##0/1> <Loss_D: 0.0626> <Loss_G: 6.7228> <D(x): 0.9996> D(G(z12)): 0.0599//0.0013
<7/2000##0/1> <Loss_D: 0.1349> <Loss_G: 9.2716> <D(x): 0.9991> D(G(z12)): 0.1156//0.0002
<8/2000##0/1> <Loss_D: 0.0281> <Loss_G: 6.8872> <D(x): 0.9973> D(G(z12)): 0.0249//0.0011
<9/2000##0/1> <Loss_D: 0.0088> <Loss_G: 7.2717> <D(x): 0.9958> D(G(z12)): 0.0045//0.0011
<10/2000##0/1> <Loss_D: 0.0167> <Loss_G: 5.8953> <D(x): 0.9966> D(G(z12)): 0.0132//0.0029
<11/2000##0/1> <Loss

KeyboardInterrupt: 